# MQ-8

## Specifications

### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  10KOhm
    Rh (Heater Resistance)     :  31 +/- 5%
    Ph (Heating consumption)   :  less than 800mw
    

### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 10kOhm  -  60kOhm (1000ppm H2)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        Hydrogen (H2):  100ppm  - 10000ppm 

In [3]:
#%pip install numpy

import numpy as np
import math

In [1]:
## Gas concentration values referenced in the datasheet

# Hydrogen
h2 = {
    'p1' : {'x': 200,   'y': 8.449},
    'p2' : {'x': 500,   'y': 2.680},
    'p3' : {'x': 800,   'y': 1.401},
    'p4' : {'x': 1000,  'y': 1.000},
    'p5' : {'x': 1500,  'y': 0.563},
    'p6' : {'x': 2000,  'y': 0.378},
    'p7' : {'x': 3000,  'y': 0.213},
    'p8' : {'x': 5000,  'y': 0.096},
    'p9' : {'x': 10000, 'y': 0.029}
}

# Liquefied Petroleum Gas
lpg = {
    'p1' : {'x': 200,   'y': 34.617},
    'p2' : {'x': 500,   'y': 23.220},
    'p3' : {'x': 800,   'y': 20.495},
    'p4' : {'x': 1000,  'y': 19.255},
    'p5' : {'x': 1500,  'y': 17.210},
    'p6' : {'x': 2000,  'y': 16.169},
    'p7' : {'x': 3000,  'y': 14.816},
    'p8' : {'x': 5000,  'y': 13.747},
    'p9' : {'x': 10000, 'y': 12.755}
}

# Methane
ch4 = {
    'p1' : {'x': 200,   'y': 53.579},
    'p2' : {'x': 500,   'y': 45.555},
    'p3' : {'x': 800,   'y': 45.555},
    'p4' : {'x': 1000,  'y': 43.881},
    'p5' : {'x': 1500,  'y': 41.744},
    'p6' : {'x': 2000,  'y': 40.210},
    'p7' : {'x': 3000,  'y': 37.778},
    'p8' : {'x': 5000,  'y': 34.188},
    'p9' : {'x': 10000, 'y': 29.433}
}

# Carbon Monoxide
co = {
    'p1' : {'x': 200,   'y': 65.421},
    'p2' : {'x': 500,   'y': 61.464},
    'p3' : {'x': 800,   'y': 59.948},
    'p4' : {'x': 1000,  'y': 56.322},
    'p5' : {'x': 1500,  'y': 54.252},
    'p6' : {'x': 2000,  'y': 52.915},
    'p7' : {'x': 3000,  'y': 49.714},
    'p8' : {'x': 5000,  'y': 45.555},
    'p9' : {'x': 10000, 'y': 40.210}
}

alcohol = {
    'p1' : {'x': 200,   'y': 24.408},
    'p2' : {'x': 500,   'y': 14.632},
    'p3' : {'x': 800,   'y': 11.119},
    'p4' : {'x': 1000,  'y': 10.446},
    'p5' : {'x': 1500,  'y': 7.938},
    'p6' : {'x': 2000,  'y': 7.095},
    'p7' : {'x': 3000,  'y': 5.391},
    'p8' : {'x': 5000,  'y': 3.996},
    'p9' : {'x': 10000, 'y': 2.366}
}


AIR = 69.634

In [4]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

In [5]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [7]:
## Test the _getPoints_ function
"""
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']
_h2  = [ h2[n]      for n in list(h2)      if n in points]


racio  = 1
p1, p2  = getPoints(racio, _h2)

m = M(p1,p2)
b = B(p1, m)

h2_ppm  = PPM(racio, m, b)
print(f"{p1}\n{p2}")
print(f"[ m: {m:.2f} || b: {b:.2f} ]")
print(f"{h2_ppm:.0f}ppm")
"""
print()

In [9]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']

_h2      = [ h2[n]      for n in list(h2)      if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]

# print(_h2)
# print(_lpg)
# print(_ch4)
# print(_co)
# print(_alcohol)

In [13]:
_ratios   = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 
             9, 8, 7, 6, 5, 4, 3, 2, 1, 
             0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 
             0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]

h2_ppm      = []
lpg_ppm     = []
ch4_ppm     = []
co_ppm      = []
alcohol_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    


#print(f"{[f'{str(i).center(6)}' for i in _ratio]}")
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))
for i in range(len(_ratios)):
    if _ratios[i] > 5:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(h2_ppm[i],3),
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       round(co_ppm[i],3),
                       round(alcohol_ppm[i],3)
                      ))
    else:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(h2_ppm[i],3),
                       "----",
                       "----",
                       "----",
                       round(alcohol_ppm[i],3)
                      ))

Ratio    H2            LPG           CH4           CO            Alcohol      
100      27.837        17.535        5.894         0.393         16.007       
90       30.278        22.33         10.687        1.848         19.33        
80       33.262        29.259        20.786        10.42         23.869       
70       37.002        39.75         44.187        74.052        30.316       
60       41.846        56.617        105.536       787.052       39.954       
50       48.4          86.027        295.532       2890.236      55.379       
40       57.833        143.549       2069.232      10295.085     82.581       
30       72.757        277.766       9154.727      50877.025     138.231      
20       100.554       873.079       59796.194     483623.347    285.711      
10       174.832       95068.043     1479017.744   22718758.364  1066.56      
9        190.167       252059.818    2408569.989   40786376.053  1246.119     
8        208.908       749724.591    4154471.163   7

In [14]:
# MQ 8 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5         #volts
sensor1['Vrl'] = 0.00001   #volts
sensor1['RL']  = 10        #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 8 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5          #volts
sensor2['Vrl'] = 0.000001   #volts
sensor2['RL']  = 10         #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        168.57   2.42     69.63   
2        94.17    1.35     69.63   


In [15]:
## PPM
sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])


sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])


sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


####

sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])


sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])


sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
          ))

Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0.28     69.63    37.16         40.23         45.51         79.98         30.6         
0.48     69.63    37.16         40.23         45.51         79.98         30.6         


In [17]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 5:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.28     69.63    37.16         40.23         45.51         79.98         30.6         
1   0.38     50.22    48.23         85.16         288.22        2808.15       54.94        
2   0.48     38.9     59.14         153.05        2480.92       12021.89      86.81        
3   0.58     31.48    70.01         248.71        7325.93       38939.32      126.81       
4   0.68     26.24    80.96         377.58        17005.9       106967.29     175.65       
5   0.78     22.35    92.03         577.43        35765.63      261018.49     234.19       
6   0.88     19.34    103.28        984.41        69846.93      582735.93     303.41       
7   0.98     16.94    114.78        1611.33       128791.53     1214350.48    384.45       
8   1.08     14.99    126.55        2838.83       226898.64     2395868.66    478.63       
9   1.18     13.37    138.64        6455.44       385280.76     4522555.41    58

In [18]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio','H2', 'LPG', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])


    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 5:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    H2            LPG           CH4           CO            Alcohol      
0   0.48     69.63    37.16         40.23         45.51         79.98         30.6         
1   0.58     56.35    43.99         65.38         150.39        998.02        44.7         
2   0.68     46.98    50.87         99.25         420.24        4176.74       61.92        
3   0.78     40.01    57.82         143.49        2066.69       10284.29      82.55        
4   0.88     34.62    64.9          199.95        4688.26       22960.17      106.95       
5   0.98     30.33    72.12         270.81        8697.92       47846.18      135.52       
6   1.08     26.84    79.51         358.58        15323.58      94398.75      168.72       
7   1.18     23.94    87.12         466.21        26019.9       178191.56     207.07       
8   1.28     21.49    94.95         669.11        42868.73      324396.62     251.19       
9   1.38     19.4     103.04        973.92        68884.65      573115.67     30